In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from bs4 import BeautifulSoup
import torchvision
from torchvision import transforms, datasets, models
import torch
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from PIL import Image
import matplotlib.pyplot as plt
from torchvision.models.detection.mask_rcnn import MaskRCNNPredictor
import matplotlib.patches as patches
import os


In [2]:
import torch
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import numpy as np
from PIL import Image
import json
import os
from sklearn.preprocessing import LabelEncoder
import torchvision
from torchvision.models.detection import FasterRCNN
from torchvision.models.detection.rpn import AnchorGenerator
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
import torch.nn as nn
import torch.nn.functional as F
from collections import OrderedDict
from torchvision.models.detection.image_list import ImageList
from torchvision.ops import MultiScaleRoIAlign



def normalize_boxes(boxes, image_width, image_height):

    boxes = boxes.clone()
    boxes[:, [0, 2]] /= image_width   # x coordinates
    boxes[:, [1, 3]] /= image_height  # y coordinates
    return boxes
    
class RADataset(Dataset):
    def __init__(self, root_dir, split='train', transform=None):
t_dir
        self.transform = transform

        with open(os.path.join(root_dir, 'train_val_split.json')) as f:
            splits = json.load(f)
            
        self.image_files = splits[split]
        
        self.df = pd.read_csv('/kaggle/input/good-df-for-arise/good_data_frame_ARISE.csv')
        
    def __len__(self):
        return len(self.image_files)
    
    def __getitem__(self, idx):
        img_name = os.path.join(self.root_dir, 'jpeg', str(self.image_files[idx]))+'.jpeg'
        image = Image.open(img_name).convert('RGB')
        
        img_width, img_height = image.size

        patient_id = self.image_files[idx]
        
        patient_joints = self.df[(self.df['patient_id'] == patient_id)]
        

        boxes = []
        jsn_scores = []
        erosion_scores = []
        joint_types = []
        
        for _, row in patient_joints.iterrows():
            joint_id = row['joint_id']
            box = row[['xcenter','ycenter','dx','dy']]
            if len(box) > 0:
                
                xcenter = float(box['xcenter'])
                ycenter = float(box['ycenter'])
                width = float(box['dx'])
                height = float(box['dy'])
                
                xmin = xcenter - width
                ymin = ycenter - height
                xmax = xcenter + width
                ymax = ycenter + height
                
                
                boxes.append([xmin, ymin, xmax, ymax])
                jsn_scores.append(int(row['jsn_score']))
                erosion_scores.append(int(row['erosion_score']))
                joint_types.append(joint_id)
        
        # Convert to tensors
        boxes = torch.as_tensor(boxes, dtype=torch.float32)
        jsn_scores = torch.as_tensor(jsn_scores, dtype=torch.int32)
        erosion_scores = torch.as_tensor(erosion_scores, dtype=torch.int32)
        joint_types = torch.as_tensor(joint_types, dtype=torch.int64)
        
        target = {
            'boxes': boxes,
            'jsn_scores': jsn_scores,
            'erosion_scores': erosion_scores,
            'labels': joint_types,
            'image_id': torch.tensor(patient_id)}
        
        if self.transform:
            image = self.transform(image)
        
        return image, target

In [3]:
path = '/kaggle/input/automated-scoring-in-rheumatoid-arthritis/dataset'
# Data transforms
transform = transforms.Compose([
    transforms.ToTensor(),
])

# Create datasets
train_dataset = RADataset(root_dir=path, split='train', transform=transform)
val_dataset = RADataset(root_dir=path, split='val', transform=transform)



In [4]:
def collate_fn(batch):
    return tuple(zip(*batch))

train_loader = DataLoader(
    train_dataset, batch_size=1, shuffle=True, num_workers=4, collate_fn=collate_fn)
val_loader = DataLoader(
    val_dataset, batch_size=1, shuffle=False, num_workers=4, collate_fn=collate_fn)

# Model

In [5]:
model = torch.load('/kaggle/input/arise/pytorch/default/1/model.pth', map_location=torch.device('cpu'))
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)


FasterRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(800,), max_size=1333, mode='bilinear')
  )
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): FrozenBatchNorm2d()
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): FrozenBatchNorm2d()
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): FrozenBatchNorm2d()
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): FrozenBatchNorm2d()
          (relu): ReLU(inplace=True)
          (downsample): Sequent

In [6]:
torch.save(model, 'model_cpu.pth')

In [7]:
# 
# for imgs, annotations in train_loader:
#     imgs = list(img.to(device) for img in imgs)
#     annotations = [{k: v.to(device) for k, v in t.items()} for t in annotations]
#     print(annotations)
#     break

# Train Model

In [8]:
# from torchvision.ops import box_iou

# num_epochs = 40
# model.to(device)
    

# #all_losses

# len_dataloader = len(train_loader)
# iou_val_loss_epoch = []
# train_loss_epoch = []
# for epoch in range(num_epochs):
     
#     i = 0
#     epoch_loss = []
#     iou_train_loss = []
#     device = 'cuda'
#     for imgs, annotations in train_loader:
#         model.train() 
#         imgs = list(img.to(device) for img in imgs)
#         annotations = [{k: v.to(device) for k, v in t.items()} for t in annotations]
#         loss_dict = model([imgs[0]], [annotations[0]])
#         losses = sum(loss for loss in loss_dict.values())
        
#         optimizer.zero_grad()
#         losses.backward()
#         optimizer.step() 
        
#         epoch_loss.append(losses.cpu().item())
        
#     scheduler.step()
    
#     train_loss_epoch.append(np.mean(epoch_loss))
    
#     val_loss = []
#     iou_val_loss = []
#     model.eval()
#     for imgs, annotations in val_loader:
#         imgs = list(img.to(device) for img in imgs)
#         annotations = [{k: v.to(device) for k, v in t.items()} for t in annotations]
#         prediction = model([imgs[0]])

                
#         ious = box_iou(prediction[0]['boxes'], annotations[0]['boxes'][prediction[0]['labels']])
#         try:
#             max_ious, _ = torch.max(ious, dim=1)
#             iou_val_loss.append(max_ious.cpu().mean().item())
#         except Exception as e:
#             iou_val_loss.append(0)
#     iou_val_loss_epoch.append(np.mean(iou_val_loss))

#     if epoch>0 and iou_val_loss_epoch[epoch]>iou_val_loss_epoch[epoch-1]:
#         torch.save(model.state_dict(), 'best_model_state_dict.pth')

#     print(f"Epoch: {epoch}, Trainloss: {train_loss_epoch[epoch]} IOU_val: {iou_val_loss_epoch[epoch]}, Learning rate: {scheduler.get_last_lr()[0]:.5f}")


In [9]:
#for imgs, annotations in val_loader:
#        imgs = list(img.to(device) for img in imgs)
#        annotations = [{k: v.to(device) for k, v in t.items()} for t in annotations]
#        break

In [10]:
#model.eval()
#preds = model(imgs)

In [11]:
def get_preds(tensor, bboxes):
    unique_values = torch.unique(tensor[0])
    if len(unique_values) <= 42:
        for i in range(42):
            if i not in unique_values:
                unique_values = torch.cat([unique_values, torch.tensor([i], dtype=torch.float32)])
                tensor = torch.cat([tensor, torch.tensor([[i], [0.1]], dtype=torch.float32)], dim=1)
                bboxes = torch.cat([bboxes,torch.tensor([[0,0,5000,5000]], dtype=torch.float32)], dim=0)
    selected_indices = []
    for val in unique_values:
        # Mask where dim=0 equals current unique value
        mask = (tensor[0] == val)
        # Get corresponding scores
        scores = tensor[1][mask]
        # Find the index of the max score among these
        max_idx = torch.argmax(scores).item()
        # Get the original index in the full tensor
        original_indices = torch.where(mask)[0]
        selected_idx = original_indices[max_idx].item()
        selected_indices.append(selected_idx)
    
    # Convert to a tensor
    selected_indices = torch.tensor(selected_indices, device=tensor.device)


    return {'boxes':bboxes[selected_indices]}

In [12]:
import os

eval_paths = os.listdir('/kaggle/input/automated-scoring-in-rheumatoid-arthritis/dataset/eval_data')

In [13]:
import cv2
def crop_image_and_save(patient_id, image, bbox, joint_id):

    width = image.shape[1]
    height = image.shape[2]

    xmin, ymin, xmax, ymax = bbox
    
    if xmin>xmax:
        print(xmin, xmax)
        xmin,xmax = xmax,xmin
        print(xmin, xmax)
    if ymin>ymax:
        ymin,ymax = ymax,ymin
        
    x1 = int(max(xmin, 1))
    y1 = int(max(ymin, 1))
    x2 = int(min(xmax, height-1))
    y2 = int(min(ymax, width-1))


        
        
    cropped_img = (image[:, y1:y2, x1:x2].permute(1,2,0).numpy()*255).astype(np.int)

    output_path = '/kaggle/working/eval_cropped/'+f"{patient_id}_{joint_id}.jpeg"
    
    try:
        cv2.imwrite(output_path, cropped_img)
    except Exception as e:
        print(x1, y1, x2, y2)
        print(image.shape)
        print(cropped_img)

In [14]:
class Eval_Dataset(Dataset):
    def __init__(self, eval_pahts, transform=None):
        self.eval_pahts = eval_pahts
        self.transform = transform
        self.root_dir = '/kaggle/input/automated-scoring-in-rheumatoid-arthritis/dataset'
            
    def __len__(self):
        return len(self.eval_pahts)
    
    def __getitem__(self, idx):
        img_name = os.path.join(self.root_dir, 'eval_data', str(self.eval_pahts[idx]))
        image = Image.open(img_name).convert('RGB')
        
        if self.transform:
            image = self.transform(image)

        return image,  int(self.eval_pahts[idx].split('.')[0])

In [15]:
eval_dataset = Eval_Dataset(eval_paths, transform)

eval_loader = DataLoader(
    eval_dataset, batch_size=1, shuffle=False, num_workers=4, collate_fn=collate_fn)

In [16]:

eval_df = pd.DataFrame(columns=['patient_id', 'joint_id', 'bbox'])

In [17]:
from tqdm import tqdm

os.makedirs('/kaggle/working/eval_cropped/', exist_ok=True)

model.eval()
for img, patient_id in tqdm(eval_loader):
    imgs = img[0].to(device)
    predictions = [{k:v.detach().cpu() for k,v in model([imgs])[0].items()}]
    imgs = imgs.detach().cpu()
    tensor = torch.stack([predictions[0]['labels'].float(), predictions[0]['scores']])
    bboxes = get_preds(tensor, predictions[0]['boxes'])
    for i, bbox in enumerate(bboxes['boxes']):
        row = {'patient_id':patient_id[0],'joint_id':i,'bbox':bbox}
        eval_df= eval_df.append(row, ignore_index=True)
        crop_image_and_save(patient_id[0], imgs, bbox, i)

    
    

  0%|          | 0/30 [00:00<?, ?it/s]/opt/conda/lib/python3.7/site-packages/torch/nn/functional.py:2854: UserWarning: The default behavior for interpolate/upsample with float scale_factor will change in 1.6.0 to align with other frameworks/libraries, and use scale_factor directly, instead of relying on the computed output size. If you wish to keep the old behavior, please set recompute_scale_factor=True. See the documentation of nn.Upsample for details. 
  warnings.warn("The default behavior for interpolate/upsample with float scale_factor will change "
/opt/conda/conda-bld/pytorch_1587428398394/work/torch/csrc/utils/python_arg_parser.cpp:756: UserWarning: This overload of nonzero is deprecated:
	nonzero(Tensor input, *, Tensor out)
Consider using one of the following signatures instead:
	nonzero(Tensor input, *, bool as_tuple)
100%|██████████| 30/30 [00:17<00:00,  1.76it/s]


In [18]:
len(eval_paths)*42

1260

In [19]:
train_df = pd.DataFrame(columns=['patient_id', 'joint_id', 'bbox'])

In [20]:
from tqdm import tqdm

model.eval()
for img, annot in tqdm(train_loader):
    patient_id = [annot[0]['image_id'].item()]
    imgs = img[0].to(device)
    predictions = [{k:v.detach().cpu() for k,v in model([imgs])[0].items()}]
    imgs = imgs.detach().cpu()
    tensor = torch.stack([predictions[0]['labels'].float(), predictions[0]['scores']])
    bboxes = get_preds(tensor, predictions[0]['boxes'])
    
    for i, bbox in enumerate(bboxes['boxes']):
        row = {'patient_id':patient_id[0],'joint_id':i,'bbox':bbox}
        train_df = train_df.append(row, ignore_index=True)
        crop_image_and_save(patient_id[0], imgs, bbox, i)


for img, annot in tqdm(val_loader):
    patient_id = [annot[0]['image_id'].item()]
    imgs = img[0].to(device)
    predictions = [{k:v.detach().cpu() for k,v in model([imgs])[0].items()}]
    imgs = imgs.detach().cpu()
    tensor = torch.stack([predictions[0]['labels'].float(), predictions[0]['scores']])
    bboxes = get_preds(tensor, predictions[0]['boxes'])
    for i, bbox in enumerate(bboxes['boxes']):
        row = {'patient_id':patient_id[0],'joint_id':i,'bbox':bbox}
        train_df = train_df.append(row, ignore_index=True)
        crop_image_and_save(patient_id[0], imgs, bbox, i)



100%|██████████| 60/60 [00:38<00:00,  1.57it/s]


In [22]:
len(os.listdir('/kaggle/input/automated-scoring-in-rheumatoid-arthritis/dataset/jpeg'))*42

12600

In [23]:
train_df

,patient_id,joint_id,bbox
0,165,0,"[tensor(1487.9550), tensor(502.1628), tensor(1..."
1,165,1,"[tensor(1632.1521), tensor(595.7759), tensor(1..."
2,165,2,"[tensor(1851.4111), tensor(848.0338), tensor(2..."
3,165,3,"[tensor(1317.0197), tensor(1897.6770), tensor(..."
4,165,4,"[tensor(1584.0186), tensor(1765.0103), tensor(..."
...,...,...,...
12595,483,37,"[tensor(474.5116), tensor(654.5651), tensor(79..."
12596,483,38,"[tensor(303.5309), tensor(631.4346), tensor(63..."
12597,483,39,"[tensor(179.4075), tensor(657.3367), tensor(47..."
12598,483,40,"[tensor(56.3359), tensor(746.2819), tensor(358..."


In [24]:
eval_df.to_csv('eval_df.csv', index=False)
train_df.to_csv('train_df.csv', index=False)

In [26]:
len(os.listdir('/kaggle/working/eval_cropped'))

13860

# Function to plot image

In [24]:
def plot_image(img_tensor, annotation):
    
    fig,ax = plt.subplots(1)
    img = img_tensor.cpu().data

    # Display the image
    ax.imshow(img.permute(1, 2, 0))
    
    for box in annotation["boxes"]:
        xmin, ymin, xmax, ymax = box

        # Create a Rectangle patch
        rect = patches.Rectangle((xmin,ymin),(xmax-xmin),(ymax-ymin),linewidth=1,edgecolor='r',facecolor='none')

        # Add the patch to the Axes
        ax.add_patch(rect)

    plt.show()